In [0]:
!pip install tensorflow==2.0.0

In [1]:
from google.colab import drive
import sys
drive.mount('/content/gdrive/',force_remount=True)

o_root = r"/content/gdrive/My Drive/ProjetoFinal"
sys.path.append(f'{o_root}/Code/source')
sys.path.append(f'/content/gdrive/My Drive/Puc/Projeto Final/Code/source')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [5]:
!7za a '/content/gdrive/My Drive/Puc/Projeto Final/Datasets/separation_40.7z' '/content/gdrive/My Drive/Puc/Projeto Final/Datasets/separation_40'


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.00GHz (50653),ASM,AES-NI)

Scanning the drive:
  0M Scan  /content/gdrive/My Drive/Puc/Projeto Final/Datasets/                                                                 0M 4 Scan  /content/gdrive/My Drive/Puc/ . asets/separation_40/tc/ch/00/                                                                           30M 115 Scan  /content/gdrive/My Drive/Puc/P . inal/Datasets/separation_40/nw/                                                                        

In [0]:
!7za x '/content/gdrive/My Drive/Puc/Projeto Final/Datasets/tuned/train_sliding.7z' -odata


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/gdrive/My Drive/Puc/Projeto Final/Datasets/tuned/                                                                    1 file, 6982455881 bytes (6659 MiB)

Extracting archive: /content/gdrive/My Drive/Puc/Projeto Final/Datasets/tuned/train_sliding.7z
--
Path = /content/gdrive/My Drive/Puc/Projeto Final/Datasets/tuned/train_sliding.7z
Type = 7z
Physical Size = 6982455881
Headers Size = 81576
Method = LZMA2:24
Solid = +
Blocks = 10

  0%      0% 225 - sliding/bc/cctv/00/cctv_0001.sliding                                               

In [0]:
!cat $(find data/ -type f) > train.sliding

In [0]:
import pandas as pd
df = pd.read_csv("train.sliding")

In [0]:
import tensorflow as tf
import os
import numpy as np
from tqdm import tqdm
import shutil
from training.train_mentions import MAX_NUM_MENTIONS, MAX_NUM_PREDICTIONS, INPUT_EMBEDDINGS_SIZE


def create_float_feature(values):
    feature = tf.train.Feature(float_list=tf.train.FloatList(value=list(values)))
    return feature


def create_int_feature(values):
    f = tf.train.Feature(int64_list=tf.train.Int64List(value=list(values)))
    return f


name_to_features = {
    "input_embeddings":
        tf.io.FixedLenFeature(MAX_NUM_MENTIONS * INPUT_EMBEDDINGS_SIZE, tf.float32),
    "input_mask":
        tf.io.FixedLenFeature(MAX_NUM_MENTIONS, tf.int64),
    "labels":
        tf.io.FixedLenFeature(MAX_NUM_PREDICTIONS, tf.int64),
    "output_mask":
        tf.io.FixedLenFeature(MAX_NUM_PREDICTIONS, tf.int64),
    "cumsum":
        tf.io.FixedLenFeature(MAX_NUM_PREDICTIONS * INPUT_EMBEDDINGS_SIZE * 5, tf.float32),
}


def to_feature(tensor):
    as_numpy = tensor.numpy()
    if as_numpy.dtype == np.float32:
        return create_float_feature(as_numpy)
    return create_int_feature(as_numpy)


def check(out_dir, ds):
    counter = 0
    for batch in tqdm(ds):
        new_file = f"{out_dir}/{counter}.tsv"
        counter += 1
        with tf.io.TFRecordWriter(new_file) as writer:
            for raw_record in batch:
                as_dict = tf.io.parse_single_example(raw_record, name_to_features)
                features = {x: to_feature(as_dict[x]) for x in as_dict}
                example = tf.train.Example(features=tf.train.Features(feature=features))
                writer.write(example.SerializeToString())


main_root = f"{o_root}/tsv/cumsum_40"
work_dir = "work"

os.makedirs(f"{work_dir}/data_in", exist_ok=True)
os.makedirs(f"{work_dir}/data_out", exist_ok=True)
for root, dirs, current_files in os.walk(main_root):
    for file_name in current_files:
        print(f"{file_name}")
        if "_" not in file_name:
            print("Ignoring")
            continue
        shutil.copy(f"{root}/{file_name}", f"{work_dir}/data_in")
        ds = tf.data.TFRecordDataset(filenames=[f"{work_dir}/data_in/{file_name}"])
        ds = ds.batch(10)
        check(f"{work_dir}/data_out", ds)

        subfolders = file_name.split("_")[:3]
        out_dir = os.path.join(main_root, *subfolders, file_name.replace(".tsv", ""))
        os.makedirs(out_dir, exist_ok=True)
        
        print(f"Moving files to {out_dir}")
        for src in os.listdir(f"{work_dir}/data_out"):
            shutil.move(f"{work_dir}/data_out/{src}", out_dir)
        os.remove(f"{work_dir}/data_in/{file_name}")
        os.remove(f"{root}/{file_name}")